In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

import cv2
from tqdm import tqdm

from multiprocessing import Pool, cpu_count
from keras.applications import ResNet50
from subprocess import check_output
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from keras.callbacks import LambdaCallback, EarlyStopping
import matplotlib.pyplot as plt
from model.resnet_152_keras import resnet152_model

Using TensorFlow backend.


In [2]:
img_size = 256
model = resnet152_model('resnet152_weights_tf.h5', input_size = img_size, include_top = False)

In [3]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
data (InputLayer)                (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2D (None, 262, 262, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 128, 128, 64)  9408                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 128, 128, 64)  256                                          
___________________________________________________________________________________________

bn2c_branch2a (BatchNormalizatio (None, 63, 63, 64)    256                                          
____________________________________________________________________________________________________
scale2c_branch2a (Scale)         (None, 63, 63, 64)    128                                          
____________________________________________________________________________________________________
res2c_branch2a_relu (Activation) (None, 63, 63, 64)    0                                            
____________________________________________________________________________________________________
res2c_branch2b_zeropadding (Zero (None, 65, 65, 64)    0                                            
____________________________________________________________________________________________________
res2c_branch2b (Conv2D)          (None, 63, 63, 64)    36864                                        
___________________________________________________________________________________________

____________________________________________________________________________________________________
scale3b1_branch2c (Scale)        (None, 32, 32, 512)   1024                                         
____________________________________________________________________________________________________
res3b1 (Add)                     (None, 32, 32, 512)   0                                            
____________________________________________________________________________________________________
res3b1_relu (Activation)         (None, 32, 32, 512)   0                                            
____________________________________________________________________________________________________
res3b2_branch2a (Conv2D)         (None, 32, 32, 128)   65536                                        
____________________________________________________________________________________________________
bn3b2_branch2a (BatchNormalizati (None, 32, 32, 128)   512                                 

res3b4_branch2c (Conv2D)         (None, 32, 32, 512)   65536                                        
____________________________________________________________________________________________________
bn3b4_branch2c (BatchNormalizati (None, 32, 32, 512)   2048                                         
____________________________________________________________________________________________________
scale3b4_branch2c (Scale)        (None, 32, 32, 512)   1024                                         
____________________________________________________________________________________________________
res3b4 (Add)                     (None, 32, 32, 512)   0                                            
____________________________________________________________________________________________________
res3b4_relu (Activation)         (None, 32, 32, 512)   0                                            
___________________________________________________________________________________________

____________________________________________________________________________________________________
res3b7_branch2b_relu (Activation (None, 32, 32, 128)   0                                            
____________________________________________________________________________________________________
res3b7_branch2c (Conv2D)         (None, 32, 32, 512)   65536                                        
____________________________________________________________________________________________________
bn3b7_branch2c (BatchNormalizati (None, 32, 32, 512)   2048                                         
____________________________________________________________________________________________________
scale3b7_branch2c (Scale)        (None, 32, 32, 512)   1024                                         
____________________________________________________________________________________________________
res3b7 (Add)                     (None, 32, 32, 512)   0                                   

...

____________________________________________________________________________________________________
scale4b22_branch2b (Scale)       (None, 16, 16, 256)   512                                          
____________________________________________________________________________________________________
res4b22_branch2b_relu (Activatio (None, 16, 16, 256)   0                                            
____________________________________________________________________________________________________
res4b22_branch2c (Conv2D)        (None, 16, 16, 1024)  262144                                       
____________________________________________________________________________________________________
bn4b22_branch2c (BatchNormalizat (None, 16, 16, 1024)  4096                                         
____________________________________________________________________________________________________
scale4b22_branch2c (Scale)       (None, 16, 16, 1024)  2048                                

res4b25_branch2b (Conv2D)        (None, 16, 16, 256)   589824                                       
____________________________________________________________________________________________________
bn4b25_branch2b (BatchNormalizat (None, 16, 16, 256)   1024                                         
____________________________________________________________________________________________________
scale4b25_branch2b (Scale)       (None, 16, 16, 256)   512                                          
____________________________________________________________________________________________________
res4b25_branch2b_relu (Activatio (None, 16, 16, 256)   0                                            
____________________________________________________________________________________________________
res4b25_branch2c (Conv2D)        (None, 16, 16, 1024)  262144                                       
___________________________________________________________________________________________

____________________________________________________________________________________________________
res4b28_branch2b_zeropadding (Ze (None, 18, 18, 256)   0                                            
____________________________________________________________________________________________________
res4b28_branch2b (Conv2D)        (None, 16, 16, 256)   589824                                       
____________________________________________________________________________________________________
bn4b28_branch2b (BatchNormalizat (None, 16, 16, 256)   1024                                         
____________________________________________________________________________________________________
scale4b28_branch2b (Scale)       (None, 16, 16, 256)   512                                          
____________________________________________________________________________________________________
res4b28_branch2b_relu (Activatio (None, 16, 16, 256)   0                                   

scale4b31_branch2a (Scale)       (None, 16, 16, 256)   512                                          
____________________________________________________________________________________________________
res4b31_branch2a_relu (Activatio (None, 16, 16, 256)   0                                            
____________________________________________________________________________________________________
res4b31_branch2b_zeropadding (Ze (None, 18, 18, 256)   0                                            
____________________________________________________________________________________________________
res4b31_branch2b (Conv2D)        (None, 16, 16, 256)   589824                                       
____________________________________________________________________________________________________
bn4b31_branch2b (BatchNormalizat (None, 16, 16, 256)   1024                                         
___________________________________________________________________________________________

res5c_branch2c (Conv2D)          (None, 8, 8, 2048)    1048576                                      
____________________________________________________________________________________________________
bn5c_branch2c (BatchNormalizatio (None, 8, 8, 2048)    8192                                         
____________________________________________________________________________________________________
scale5c_branch2c (Scale)         (None, 8, 8, 2048)    4096                                         
____________________________________________________________________________________________________
res5c (Add)                      (None, 8, 8, 2048)    0                                            
____________________________________________________________________________________________________
res5c_relu (Activation)          (None, 8, 8, 2048)    0                                            
___________________________________________________________________________________________

In [5]:
from keras.layers import Add
adds = []
for i, l in enumerate(model.layers):
    if type(l) is Add:
        adds.append(i)
np.array(adds)

array([ 22,  36,  50,  67,  81,  95, 109, 123, 137, 151, 165, 182, 196,
       210, 224, 238, 252, 266, 280, 294, 308, 322, 336, 350, 364, 378,
       392, 406, 420, 434, 448, 462, 476, 490, 504, 518, 532, 546, 560,
       574, 588, 602, 616, 630, 644, 658, 672, 689, 703, 717])

In [6]:
#Adding classifier
x = model.output
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
x = Dense(17, activation="sigmoid")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = x)
model_final.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
data (InputLayer)                (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2D (None, 262, 262, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 128, 128, 64)  9408                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 128, 128, 64)  256                                          
___________________________________________________________________________________________

bn2c_branch2a (BatchNormalizatio (None, 63, 63, 64)    256                                          
____________________________________________________________________________________________________
scale2c_branch2a (Scale)         (None, 63, 63, 64)    128                                          
____________________________________________________________________________________________________
res2c_branch2a_relu (Activation) (None, 63, 63, 64)    0                                            
____________________________________________________________________________________________________
res2c_branch2b_zeropadding (Zero (None, 65, 65, 64)    0                                            
____________________________________________________________________________________________________
res2c_branch2b (Conv2D)          (None, 63, 63, 64)    36864                                        
___________________________________________________________________________________________

____________________________________________________________________________________________________
scale3b1_branch2c (Scale)        (None, 32, 32, 512)   1024                                         
____________________________________________________________________________________________________
res3b1 (Add)                     (None, 32, 32, 512)   0                                            
____________________________________________________________________________________________________
res3b1_relu (Activation)         (None, 32, 32, 512)   0                                            
____________________________________________________________________________________________________
res3b2_branch2a (Conv2D)         (None, 32, 32, 128)   65536                                        
____________________________________________________________________________________________________
bn3b2_branch2a (BatchNormalizati (None, 32, 32, 128)   512                                 

res3b4_branch2c (Conv2D)         (None, 32, 32, 512)   65536                                        
____________________________________________________________________________________________________
bn3b4_branch2c (BatchNormalizati (None, 32, 32, 512)   2048                                         
____________________________________________________________________________________________________
scale3b4_branch2c (Scale)        (None, 32, 32, 512)   1024                                         
____________________________________________________________________________________________________
res3b4 (Add)                     (None, 32, 32, 512)   0                                            
____________________________________________________________________________________________________
res3b4_relu (Activation)         (None, 32, 32, 512)   0                                            
___________________________________________________________________________________________

____________________________________________________________________________________________________
res3b7_branch2b_relu (Activation (None, 32, 32, 128)   0                                            
____________________________________________________________________________________________________
res3b7_branch2c (Conv2D)         (None, 32, 32, 512)   65536                                        
____________________________________________________________________________________________________
bn3b7_branch2c (BatchNormalizati (None, 32, 32, 512)   2048                                         
____________________________________________________________________________________________________
scale3b7_branch2c (Scale)        (None, 32, 32, 512)   1024                                         
____________________________________________________________________________________________________
res3b7 (Add)                     (None, 32, 32, 512)   0                                   

...

res4b25_branch2b (Conv2D)        (None, 16, 16, 256)   589824                                       
____________________________________________________________________________________________________
bn4b25_branch2b (BatchNormalizat (None, 16, 16, 256)   1024                                         
____________________________________________________________________________________________________
scale4b25_branch2b (Scale)       (None, 16, 16, 256)   512                                          
____________________________________________________________________________________________________
res4b25_branch2b_relu (Activatio (None, 16, 16, 256)   0                                            
____________________________________________________________________________________________________
res4b25_branch2c (Conv2D)        (None, 16, 16, 1024)  262144                                       
___________________________________________________________________________________________

____________________________________________________________________________________________________
res4b28_branch2b_zeropadding (Ze (None, 18, 18, 256)   0                                            
____________________________________________________________________________________________________
res4b28_branch2b (Conv2D)        (None, 16, 16, 256)   589824                                       
____________________________________________________________________________________________________
bn4b28_branch2b (BatchNormalizat (None, 16, 16, 256)   1024                                         
____________________________________________________________________________________________________
scale4b28_branch2b (Scale)       (None, 16, 16, 256)   512                                          
____________________________________________________________________________________________________
res4b28_branch2b_relu (Activatio (None, 16, 16, 256)   0                                   

scale4b31_branch2a (Scale)       (None, 16, 16, 256)   512                                          
____________________________________________________________________________________________________
res4b31_branch2a_relu (Activatio (None, 16, 16, 256)   0                                            
____________________________________________________________________________________________________
res4b31_branch2b_zeropadding (Ze (None, 18, 18, 256)   0                                            
____________________________________________________________________________________________________
res4b31_branch2b (Conv2D)        (None, 16, 16, 256)   589824                                       
____________________________________________________________________________________________________
bn4b31_branch2b (BatchNormalizat (None, 16, 16, 256)   1024                                         
___________________________________________________________________________________________

____________________________________________________________________________________________________
bn4b34_branch2a (BatchNormalizat (None, 16, 16, 256)   1024                                         
____________________________________________________________________________________________________
scale4b34_branch2a (Scale)       (None, 16, 16, 256)   512                                          
____________________________________________________________________________________________________
res4b34_branch2a_relu (Activatio (None, 16, 16, 256)   0                                            
____________________________________________________________________________________________________
res4b34_branch2b_zeropadding (Ze (None, 18, 18, 256)   0                                            
____________________________________________________________________________________________________
res4b34_branch2b (Conv2D)        (None, 16, 16, 256)   589824                              

scale5a_branch2c (Scale)         (None, 8, 8, 2048)    4096                                         
____________________________________________________________________________________________________
scale5a_branch1 (Scale)          (None, 8, 8, 2048)    4096                                         
____________________________________________________________________________________________________
res5a (Add)                      (None, 8, 8, 2048)    0                                            
____________________________________________________________________________________________________
res5a_relu (Activation)          (None, 8, 8, 2048)    0                                            
____________________________________________________________________________________________________
res5b_branch2a (Conv2D)          (None, 8, 8, 512)     1048576                                      
___________________________________________________________________________________________